### 40x 이미지 Train Dataset 생성 코드
* Create_TestSet_Final.ipynb 파일에서, 하나씩 이미지 하지 않고 폴더 내 이미지 모두 돌리는 것으로 수정(220821)
* 이미지 전처리 및 컨투어
* Green, Red 처리 이미지로 세포 좌표 받아와 원본 이미지에서 크롭함

In [1]:
import numpy as np
import cv2, sys
from matplotlib import pyplot as plt
import glob
from PIL import Image

# optical, red, green 이미지(Train용) 경로 설정

In [2]:
image_optical = glob.glob('train/O*.tif')
image_red = glob.glob('train/R*.tif')
image_green = glob.glob('train/G*.tif')

In [3]:
image_optical.sort()
image_red.sort()
image_green.sort()

In [7]:
image_optical, image_red, image_green

(['train/O1.tif',
  'train/O10.tif',
  'train/O11.tif',
  'train/O12.tif',
  'train/O2.tif',
  'train/O3.tif',
  'train/O4.tif',
  'train/O5.tif',
  'train/O6.tif',
  'train/O7.tif',
  'train/O8.tif',
  'train/O9.tif'],
 ['train/R1.tif',
  'train/R10.tif',
  'train/R11.tif',
  'train/R12.tif',
  'train/R2.tif',
  'train/R3.tif',
  'train/R4.tif',
  'train/R5.tif',
  'train/R6.tif',
  'train/R7.tif',
  'train/R8.tif',
  'train/R9.tif'],
 ['train/G1.tif',
  'train/G10.tif',
  'train/G11.tif',
  'train/G12.tif',
  'train/G2.tif',
  'train/G3.tif',
  'train/G4.tif',
  'train/G5.tif',
  'train/G6.tif',
  'train/G7.tif',
  'train/G8.tif',
  'train/G9.tif'])

In [21]:
#image_optical = glob.glob('./220727_Train/optical_image/*.tif')
#image_red = glob.glob('./220727_Train/red_image/*.tif')
#image_green = glob.glob('./220727_Train/green_image/*.tif')

count=0
kernel=np.ones((120,120),np.uint8)

for i in range(len(image_red)):
    tmp_red = Image.open(image_red[i])
    tmp_red_array=np.asarray(tmp_red)
    
    tmp_red_rgb = cv2.cvtColor(tmp_red_array,cv2.COLOR_BGR2RGB)
    
    ret, dst=cv2.threshold(tmp_red_rgb,127,255,cv2.THRESH_BINARY)
    dilation_edgedR=cv2.dilate(dst,kernel,iterations=1)
    
    blurR=cv2.GaussianBlur(dilation_edgedR, ksize=(7,7), sigmaX=0)
    #ret, thresh1=cv2.threshold(blurR,127,255,cv2.THRESH_BINARY)
    
    normR=cv2.normalize(blurR,None,0,255,cv2.NORM_MINMAX)
    edgedR=cv2.Canny(normR,10,30)
    
    # kernel 재설정
    kernel=np.ones((20,20),np.uint8)
    dilation_edged2_R=cv2.dilate(edgedR,kernel,iterations=1)
    
    # kernel 재설정2
    kernel=cv2.getStructuringElement(cv2.MORPH_RECT,(7,7))
    closed_R=cv2.morphologyEx(dilation_edged2_R,cv2.MORPH_CLOSE,kernel)
    
    contours, _=cv2.findContours(closed_R.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    total=0
    
    contours_xy=np.array(contours,dtype=object,)
    contours_xy.shape
    
    # contour 후 crop
    count=0 #contour 처리한 총 이미지 개수를 알기 위함
    tmp_optical = Image.open(image_optical[i])
    tmp_optical_array = np.asarray(tmp_optical)
    
    for c in contours:
        x,y,w,h=cv2.boundingRect(c)
        print(x,y,w,h)
        if(w>0 and h>0):
            img_trim = tmp_optical_array[y:y+h+8, x:x+w+8]
            #img_trim=tmp_optical[y:y+h, x:x+w]
            cv2.imwrite(f'Train/Red/{i}/{count}.jpg',img_trim)
            count+=1
    print(count)

334 1247 189 188
882 1080 582 373
0 584 303 586
755 414 191 193
253 281 197 193
663 136 182 189
6
752 1437 83 27
704 1353 74 63
527 1255 65 62
426 1132 67 69
848 1019 62 67
935 617 157 112
645 595 69 68
274 357 94 99
319 178 106 103
9
334 1302 72 71
1041 1295 80 84
455 1204 76 63
1094 781 72 72
706 715 66 73
150 694 67 70
447 551 69 71
253 479 67 71
333 423 80 77
601 372 148 98
544 150 66 67
630 0 73 53
279 0 53 34
13
1060 1363 60 68
665 1023 117 110
896 878 59 65
1076 776 57 66
584 735 64 69
1050 491 64 65
1249 423 58 62
884 340 63 63
509 148 79 79
0 87 57 57
10
779 1319 73 67
1188 1290 59 61
1424 1191 40 57
616 1175 70 73
716 1126 70 60
534 1052 68 64
128 743 66 63
1052 594 62 66
860 243 155 78
98 202 60 61
10
595 1375 55 68
815 1108 62 59
1079 1053 46 46
1306 995 60 59
1329 900 59 71
1032 729 73 69
905 689 105 74
804 682 65 60
485 669 57 62
1102 416 85 87
896 273 79 70
1173 237 66 56
801 218 60 70
282 192 51 49
87 0 111 85
15
569 1447 46 17
1360 1344 43 47
719 1147 70 68
772 941 53 

In [22]:
#green에 대해서 contour

for i in range(len(image_green)):
    tmp_green = Image.open(image_green[i])
    tmp_green_array=np.asarray(tmp_green)
    
    tmp_green_rgb = cv2.cvtColor(tmp_green_array,cv2.COLOR_BGR2RGB)
    
    ret, dst=cv2.threshold(tmp_green_rgb,127,255,cv2.THRESH_BINARY)
    dilation_edgedG=cv2.dilate(dst,kernel,iterations=1)
    
    blurG=cv2.GaussianBlur(dilation_edgedG, ksize=(7,7), sigmaX=0)
    #ret, thresh1=cv2.threshold(blurR,127,255,cv2.THRESH_BINARY)
    
    normG=cv2.normalize(blurG,None,0,255,cv2.NORM_MINMAX)
    edgedG=cv2.Canny(normG,10,30)
    
    # kernel 재설정
    kernel=np.ones((20,20),np.uint8)
    dilation_edged2_G=cv2.dilate(edgedG,kernel,iterations=1)
    
    # kernel 재설정2
    kernel=cv2.getStructuringElement(cv2.MORPH_RECT,(7,7))
    closed_G=cv2.morphologyEx(dilation_edged2_G,cv2.MORPH_CLOSE,kernel)
    
    contours, _=cv2.findContours(closed_G.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    total=0
    
    contours_xy=np.array(contours,dtype=object,)
    contours_xy.shape
    
    # contour 후 crop
    count=0 #contour 처리한 총 이미지 개수를 알기 위함
    tmp_optical = Image.open(image_optical[i])
    tmp_optical_array = np.asarray(tmp_optical)
    
    for c in contours:
        x,y,w,h=cv2.boundingRect(c)
        #print(x,y,w,h)
        if(w>0 and h>0):
            img_trim = tmp_optical_array[y:y+h+8, x:x+w+8]
            cv2.imwrite(f'Train/Green/{i}/{count}.jpg',img_trim)
            count+=1

In [14]:
print(count)

0
